In [8]:
import pandas as pd
import jieba
from tqdm import tqdm
import matplotlib.pyplot as plt
import numpy as np
from collections import Counter

pd.set_option('display.max_colwidth', None)#setting the maximize string show

fb = pd.read_csv('nysu_10902_2019_research_right.csv')
politics = pd.read_csv('9th_legislator_promise.csv')
fb.page_name.unique() #觀察人名在資料集中叫什麼名字
politics.姓名.unique()

array(['吳思瑤', '姚文智', '何志偉', '蔣萬安', '李彥秀', '林昶佐', '蔣乃辛', '費鴻泰', '賴士葆',
       '呂 孫 綾', '林 淑 芬', '高志鵬', '余天', '吳 秉 叡', '蘇 巧 慧', '張 宏 陸', '羅 致 政',
       '江 永 昌', '林 德 福', '吳 琪 銘', '羅 明 才', '黃 國 昌', '蔡適應', '鄭 運 鵬',
       '陳賴素美', '陳學聖', '鄭寶清', '呂玉玲', '趙正宇', '林為洲', '柯建銘', '陳超明', '徐志榮',
       '蔡其昌', '顏寬恒', '洪慈庸', '張廖萬堅', '盧秀燕', '沈智慧', '黃國書', '何欣純', '江啟臣',
       '王惠美', '柯呈枋', '黃秀芳', '洪宗熠', '陳素月', '馬 文 君', '許 淑 華', '蘇 治 芬',
       '劉 建 國', '李 俊 俋', '蔡 易 餘', '陳 明 文', '葉宜津', '黃偉哲', '郭國文  ', '陳亭妃',
       '林俊憲', '王定宇', '邱 議 瑩', '邱 志 偉', '劉 世 芳', '林 岱 樺', '管 碧 玲', '李 昆 澤',
       '趙 天 麟', '馬 凱 妮', '賴 瑞 隆', '蘇震清', '鍾佳濱', '莊瑞雄', '陳歐珀', '蕭美琴',
       '劉櫂豪', '楊 曜', '楊鎮浯', '陳玉珍', '陳雪生', '廖國棟', '鄭天財', '陳瑩', '高金素梅',
       '簡東明', '孔文吉', nan], dtype=object)

In [9]:
lo = fb[fb.page_name == "羅致政粉絲團"]
lo.head(1)

,post_id,page_id,page_name,page_category,message,story,type,permalink,created_time_gmt,created_time_taipei,...,link_name,link_description,like_count,haha_count,love_count,angry_count,sad_count,wow_count,comment_count,share_count
23477,182836328407031_2219870311370279,182836328407031,羅致政粉絲團,政治人物,違法兼職，事證明確，卡不卡管，台大自斷,NaN,link,https://www.facebook.com/182836328407031/posts/2219870311370279/,2019-01-15T06:40:53+0000,2019-01-15T14:40:53,...,管中閔違法兼職 監察院7：4通過彈劾 | 蘋果日報,台大校長管中閔已在上周上任，不過，對於他過去擔任學校主管及政府部門任職時在外兼職違法事宜，監察院今天召開管中閔兼職的彈劾審查，結果以7：4通...,1029,0,0,0,0,0,42,29


In [10]:
lo['new_date'] = pd.to_datetime(lo['created_time_taipei']).dt.date
lo['month_year'] = pd.to_datetime(lo['new_date']).dt.to_period('M')
lo.head(1)

<ipython-input-10-067a578daaf3>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  lo['new_date'] = pd.to_datetime(lo['created_time_taipei']).dt.date
<ipython-input-10-067a578daaf3>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  lo['month_year'] = pd.to_datetime(lo['new_date']).dt.to_period('M')


,post_id,page_id,page_name,page_category,message,story,type,permalink,created_time_gmt,created_time_taipei,...,like_count,haha_count,love_count,angry_count,sad_count,wow_count,comment_count,share_count,new_date,month_year
23477,182836328407031_2219870311370279,182836328407031,羅致政粉絲團,政治人物,違法兼職，事證明確，卡不卡管，台大自斷,NaN,link,https://www.facebook.com/182836328407031/posts/2219870311370279/,2019-01-15T06:40:53+0000,2019-01-15T14:40:53,...,1029,0,0,0,0,0,42,29,2019-01-15,2019-01


In [11]:
# 依照按讚數 低到高排序
lo = lo.sort_values(by=['like_count'])
# 用新的排序當作順序
lo.reset_index(inplace=True)

結巴斷詞 jieba

len(lo): 看羅致政的貼文總共有幾筆
list(lo['message']): 把要斷詞的內容轉成 list 格式
創造一個新的dataframe叫做lo_docs，型態是pandas.core.frame.DataFrame
原本liu_docs裡面的'jieba_results'欄位型態是pandas.core.series.Series，需要轉成'str'(字串)型態才能做後續處理。

In [12]:
doc_names = range(len(lo)) #有幾篇文章 : 444 篇文章
doc_names

range(0, 871)

In [13]:
lo = lo.replace(np.nan, '', regex=True) # 將nan取代成空白字串
text_list = list(lo['message']) # 把所有message轉成一個list 用for iterate

In [17]:
# 先看第一個
words = jieba.lcut(text_list[1], cut_all = False) # lcut return a "list"
print(words)

['每年', '端午', '節', '，', '都', '會', '被', '問到', '「', '北部', '粽', '和', '南部', '粽', '你', '喜歡', '吃', '哪個', '？', '」', '最近', '連冰', '粽和住', '海邊', '的', 'XX', '粽', '（', '愛惜', '生命', '，', '大家', '知道', '就', '好', '）', '也', '參戰', '！', '\r\n', '\r\n', '#', '我小時候', '最常', '吃鹼', '粽', '沾', '砂糖', '\r\n', '\r\n', '你們', '最想', '吃', '哪個', '？', '請', '留言', '告訴', '我', '！']
